In [2]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from datetime import datetime
import math
from collections import defaultdict

### Creating Play Data for Display

This notebook creates datasets of time series data for each play in the NFL Big Data Bowl dataset.

In [3]:
# get all game data
game_data = pd.read_csv("../../NFLData/all_games.csv")

In [4]:
# get tracking data from all games 

tracking_w1_data_original = pd.read_csv('../../NFLData/week1.csv')
tracking_w2_data_original = pd.read_csv('../../NFLData/week2.csv')
tracking_w3_data_original = pd.read_csv('../../NFLData/week3.csv')
tracking_w4_data_original = pd.read_csv('../../NFLData/week4.csv')
tracking_w5_data_original = pd.read_csv('../../NFLData/week5.csv')
tracking_w6_data_original = pd.read_csv('../../NFLData/week6.csv')
tracking_w7_data_original = pd.read_csv('../../NFLData/week7.csv')
tracking_w8_data_original = pd.read_csv('../../NFLData/week8.csv')
tracking_data = pd.concat([tracking_w1_data_original,tracking_w2_data_original,tracking_w3_data_original,tracking_w4_data_original,tracking_w5_data_original,tracking_w6_data_original,tracking_w7_data_original,tracking_w8_data_original])

In [5]:
# get tracking data from 2018 (largest dataset)

tracking_w1_data_2018 = pd.read_csv('../../NFLData/2018week1.csv')
tracking_w2_data_2018 = pd.read_csv('../../NFLData/2018week2.csv')
tracking_w3_data_2018 = pd.read_csv('../../NFLData/2018week3.csv')
tracking_w4_data_2018 = pd.read_csv('../../NFLData/2018week4.csv')
tracking_w5_data_2018 = pd.read_csv('../../NFLData/2018week5.csv')
tracking_w6_data_2018 = pd.read_csv('../../NFLData/2018week6.csv')
tracking_w7_data_2018 = pd.read_csv('../../NFLData/2018week7.csv')
tracking_w8_data_2018 = pd.read_csv('../../NFLData/2018week8.csv')
tracking_w9_data_2018 = pd.read_csv('../../NFLData/2018week9.csv')
tracking_w10_data_2018 = pd.read_csv('../../NFLData/2018week10.csv')
tracking_w11_data_2018 = pd.read_csv('../../NFLData/2018week11.csv')
tracking_w12_data_2018 = pd.read_csv('../../NFLData/2018week12.csv')
tracking_w13_data_2018 = pd.read_csv('../../NFLData/2018week13.csv')
tracking_w14_data_2018 = pd.read_csv('../../NFLData/2018week14.csv')
tracking_w15_data_2018 = pd.read_csv('../../NFLData/2018week15.csv')
tracking_w16_data_2018 = pd.read_csv('../../NFLData/2018week16.csv')
tracking_w17_data_2018 = pd.read_csv('../../NFLData/2018week17.csv')
tracking_2018_data = pd.concat([tracking_w1_data_2018,tracking_w2_data_2018,tracking_w3_data_2018,tracking_w4_data_2018,
                           tracking_w5_data_2018,tracking_w6_data_2018,tracking_w7_data_2018,tracking_w8_data_2018,
                           tracking_w9_data_2018,tracking_w10_data_2018,tracking_w11_data_2018,tracking_w12_data_2018,
                           tracking_w13_data_2018,tracking_w14_data_2018,tracking_w15_data_2018,tracking_w16_data_2018,
                           tracking_w17_data_2018])

In [6]:
# select compatible columns
tracking_2018_data = tracking_2018_data[["gameId","playId","nflId","frameId","time","jerseyNumber","team","playDirection","x","y","s","a"
                                         ,"dis","o","dir","event"]]

# concatenate all data
tracking_data = pd.concat([tracking_data,tracking_2018_data])

In [7]:
# get all teams in the dataset
teams = game_data[["gameId","homeTeamAbbr","visitorTeamAbbr"]]
display(teams)

,gameId,homeTeamAbbr,visitorTeamAbbr
0,2021090900,TB,DAL
1,2021091200,ATL,PHI
2,2021091201,BUF,PIT
3,2021091202,CAR,NYJ
4,2021091203,CIN,MIN
...,...,...,...
370,2018123013,DEN,LAC
371,2018123015,SEA,ARI
372,2018123014,LA,SF
373,2018122200,TEN,WAS


In [8]:
print(tracking_data.shape[0])

26623566


In [9]:
# drop null values
tracking_data= tracking_data.dropna()

In [10]:
print(tracking_data.shape[0])

25014220


In [11]:
# merge on game data
tracking_data = tracking_data.merge(game_data,on="gameId",how="left")
display(tracking_data)

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,...,a,dis,o,dir,event,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr
0,2021090900,97,25511.0,1,2021-09-10T00:26:31.100,12.0,TB,right,37.77,24.22,...,0.30,0.03,165.16,84.99,None,1,09/09/2021,20:20:00,TB,DAL
1,2021090900,97,25511.0,2,2021-09-10T00:26:31.200,12.0,TB,right,37.78,24.22,...,0.11,0.02,164.33,92.87,None,1,09/09/2021,20:20:00,TB,DAL
2,2021090900,97,25511.0,3,2021-09-10T00:26:31.300,12.0,TB,right,37.78,24.24,...,0.10,0.01,160.24,68.55,None,1,09/09/2021,20:20:00,TB,DAL
3,2021090900,97,25511.0,4,2021-09-10T00:26:31.400,12.0,TB,right,37.73,24.25,...,0.24,0.06,152.13,296.85,None,1,09/09/2021,20:20:00,TB,DAL
4,2021090900,97,25511.0,5,2021-09-10T00:26:31.500,12.0,TB,right,37.69,24.26,...,0.18,0.04,148.33,287.55,None,1,09/09/2021,20:20:00,TB,DAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25014215,2018123011,4261,2558060.0,102,2018-12-31T04:23:24.200Z,29.0,away,left,72.95,23.20,...,0.60,0.40,110.13,105.63,None,17,12/30/2018,20:20:00,TEN,IND
25014216,2018123011,4261,2558942.0,102,2018-12-31T04:23:24.200Z,86.0,home,left,95.77,11.20,...,1.75,0.73,103.97,118.54,None,17,12/30/2018,20:20:00,TEN,IND
25014217,2018123011,4261,2558976.0,102,2018-12-31T04:23:24.200Z,23.0,away,left,101.94,14.60,...,2.24,0.57,139.23,119.97,None,17,12/30/2018,20:20:00,TEN,IND
25014218,2018123011,4261,2560789.0,102,2018-12-31T04:23:24.200Z,53.0,away,left,102.68,0.94,...,1.67,0.24,164.79,152.29,None,17,12/30/2018,20:20:00,TEN,IND


In [12]:

tracking_copy = tracking_data.copy()
categorical_data = ["playDirection","event"]
category_codes = {}

# encode categorical/text data into values for model interpretation
for i in tracking_data:
    if i in categorical_data:
        specific_codes = {}
        tracking_data[i] = tracking_data[i].astype("category")
        tracking_data[i] = tracking_data[i].cat.codes
        coded = tracking_data[i].unique()#
        count = 0
        for j in tracking_copy[i].unique():
           specific_codes[j] = coded[count]
           count += 1
        category_codes[i] = specific_codes
for i in category_codes:
  print("{} -".format(i))
  for j in category_codes[i]:
    print("{}: {}".format(j, category_codes[i][j]))

playDirection -
right: 1
left: 0
event -
None: 0
ball_snap: 4
autoevent_passforward: 2
pass_forward: 21
autoevent_ballsnap: 1
line_set: 17
play_action: 31
pass_arrived: 20
autoevent_passinterrupted: 3
fumble: 10
fumble_offense_recovered: 12
qb_sack: 33
run: 36
man_in_motion: 18
pass_outcome_caught: 23
pass_outcome_incomplete: 24
pass_tipped: 28
qb_strip_sack: 35
shift: 39
first_contact: 9
huddle_break_offense: 14
lateral: 16
handoff: 13
penalty_flag: 30
tackle: 41
dropped_pass: 5
out_of_bounds: 19
pass_outcome_interception: 25
pass_shovel: 27
touchdown: 44
fumble_defense_recovered: 11
pass_outcome_touchdown: 26
touchback: 43
penalty_accepted: 29
qb_spike: 34
field_goal_blocked: 6
punt_fake: 32
snap_direct: 40
run_pass_option: 37
pass_lateral: 22
field_goal_fake: 7
huddle_start_offense: 15
timeout_home: 42
safety: 38
field_goal_play: 8


In [13]:
event_dict = category_codes['event']
# create array of terminal events that result in a play ending
terminal_events = ['qb_sack','pass_outcome_incomplete','qb_strip_sack','pass_outcome_caught','pass_tipped','fumble','fumble_offense_recovered','lateral','run']

In [14]:
# function to generate player sequences from tracking dataframe
def generate_sequences(df):

    sequence_dict_dense = defaultdict(dict)
    current_seq_dense = []
    gameID = -1
    playID = -1
    nflID = -1
    team = -1
    collect = False
    terminal = False
    # for each row (frame in play)
    for row in df.itertuples():
        # check if frames are being collected
        if not(collect):
            # set collect to true if ball snap event (4)
            collect = (row.event == 4)
        # check if there has not been a terminal event
        if not(terminal):
            # check if the event on the current frame is a terminal event
            terminal= (row.event in terminal_events)    
        # check if the play has changed
        if (gameID == -1 or playID == -1 or nflID == -1) or (gameID != row.gameId or playID != row.playId or nflID != row.nflId):
            # check if the collected play should be added to all sequences
            if len(current_seq_dense) > 10:
                # add to sequence dictionary with game, play, team and NFL ID keys
                sequence_dict_dense[(gameID,playID,team)][nflID] = current_seq_dense
                current_seq_dense = []

            # reset flags and key values
            terminal = (row.event in terminal_events)
            collect = (row.event == 4)
            gameID = row.gameId
            playID = row.playId
            nflID = row.nflId
            # check the team for the current row and replace the row value with the actual team abbreviations
            if row.team == "home":
                team = row.homeTeamAbbr
            elif row.team == "away":
                team = row.visitorTeamAbbr
            else:
                team = row.team
            # if we are collecting frames for the new play, capture the current frame
            if collect and len(current_seq_dense) < 90 and not(terminal):
                current_seq_dense.append([row.x,row.y]) 
        # if we are collecting frames for the current play, capture the current frame
        elif gameID == row.gameId and playID == row.playId and nflID == row.nflId and collect and not(terminal):
            current_seq_dense.append([row.x,row.y]) 
    return sequence_dict_dense

In [15]:
trajectory_dict = generate_sequences(tracking_data)

In [16]:
print(f"Sequences Generated for {len(trajectory_dict)} unique plays")

Sequences Generated for 39197 unique plays


In [17]:
trajectory_dict_keys = list(trajectory_dict.keys())
print(trajectory_dict_keys[:5])

[(2021090900, 97, 'TB'), (2021090900, 97, 'DAL'), (2021090900, 137, 'TB'), (2021090900, 137, 'DAL'), (2021090900, 187, 'TB')]


In [18]:
last_five = list(trajectory_dict.items())[-40:]

# Print the last 5 key-value pairs
for key, value in last_five:
    print(key, value)

(2018123011, 1784, 'TEN') {2558005.0: [[39.97, 26.39], [44.01, 35.63], [45.55, 12.07], [49.55, 29.2], [28.55, 23.29], [46.08, 35.21], [43.9, 11.85], [34.04, 33.63], [43.13, 20.84], [46.56, 20.9], [40.14, 35.13]]}
(2018123011, 1806, 'TEN') {2539320.0: [[36.92, 42.57], [45.63, 43.75], [43.47, 33.03], [49.59, 26.69], [43.94, 24.04], [38.48, 31.24], [43.91, 35.3], [45.67, 9.69], [45.87, 44.73], [49.49, 29.52], [26.94, 26.55]]}
(2018123011, 1828, 'TEN') {2354.0: [[45.35, 36.05], [44.05, 9.78], [35.87, 36.33], [44.1, 18.94], [46.41, 18.48], [40.21, 35.34], [37.77, 43.86], [44.13, 26.06], [49.85, 32.22], [44.38, 24.2], [43.18, 29.6]], 2558005.0: [[44.24, 35.95], [45.57, 9.44], [45.74, 16.78], [49.45, 29.77], [29.09, 24.12], [45.2, 36.62], [38.94, 9.47], [32.23, 37.04], [36.44, 17.85], [46.33, 21.37], [39.72, 29.09]]}
(2018123011, 1884, 'IND') {2557961.0: [[43.81, 37.71], [43.43, 27.18], [49.85, 32.62], [44.23, 24.7], [30.16, 26.63], [30.12, 28.89], [37.25, 14.89], [34.11, 35.63], [33.85, 15.9

In [21]:
# code to display a play's trajectories 
index = int(input("Enter an integer to select a play from the available dictionary: "))
play_key = trajectory_dict_keys[index]

print(play_key,end="\n\n")

play = trajectory_dict[play_key]
play_seq_list = list(play.values())
figure = plt.figure(figsize=(20,10))

for seq in play_seq_list:
    x = [x[0] for x in seq]
    y = [x[1] for x in seq]


    plt.scatter(x[0],y[0],color="r")
    plt.plot(x,y,color = "r")

plt.ylim([-53.3, 53.3])
plt.xlim([-120, 120])
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

ValueError: invalid literal for int() with base 10: ''

In [22]:
trajectory_dict_keys_df = pd.DataFrame(trajectory_dict_keys,columns=['gameId','playId','team'])
print(trajectory_dict_keys_df.head(5))
print(trajectory_dict_keys_df.shape[0])

       gameId  playId team
0  2021090900      97   TB
1  2021090900      97  DAL
2  2021090900     137   TB
3  2021090900     137  DAL
4  2021090900     187   TB
39197


In [23]:
trajectory_dict_keys_df.to_csv('../data/trajectory_dict_keys.csv', index=False)


In [24]:
import json

# store all plays in a JSON file

trajectory_dict_upload = {str(key): value for key, value in trajectory_dict.items()}

with open('../data/all_trajectory_dict.json', 'w') as f:
    json.dump(trajectory_dict_upload, f)